In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from ray import tune
from ray.tune.schedulers import PopulationBasedTraining
from ray.air.integrations.wandb import WandbLoggerCallback, setup_wandb

# Define the custom network structure
class CustomNet(nn.Module):
    def __init__(self, input_size, hidden_layers, num_classes):
        super(CustomNet, self).__init__()
        layers = [nn.Linear(input_size, hidden_layers[0]), nn.ReLU()]
        for i in range(len(hidden_layers) - 1):
            layers += [nn.Linear(hidden_layers[i], hidden_layers[i + 1]), nn.ReLU()]
        layers.append(nn.Linear(hidden_layers[-1], num_classes))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

# Training and evaluation logic
def train_model(config):
    wandb = setup_wandb(config, project="project_name")
    net = CustomNet(config["input_size"], config["hidden_layers"], config["num_classes"])
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=config["lr"])

    for epoch in range(config["num_epochs"]):
        inputs = torch.from_numpy(config["X_train"])
        labels = torch.from_numpy(config["y_train"])
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        accuracy = evaluate_model(net, config["X_test"], config["y_test"])
        wandb.log({"epoch": epoch, "loss": loss.item(), "accuracy": accuracy})
        tune.report(accuracy=accuracy)

# Mutation function for PBT
def mutate_layers(config):
    # Example mutation logic - can be adjusted
    new_layers = config["hidden_layers"]
    if np.random.rand() < 0.5 and len(new_layers) > 1:
        new_layers.pop()
    else:
        new_layers.append(np.random.choice([32, 64, 128]))
    return {"hidden_layers": new_layers}

# PBT Setup
scheduler = PopulationBasedTraining(
    time_attr="training_iteration",
    perturbation_interval=5,
    hyperparam_mutations={
        "lr": tune.loguniform(1e-4, 1e-1),
        "hidden_layers": mutate_layers  
    }
)

# Dummy dataset
X_train = np.random.rand(1000, 784).astype(np.float32)
y_train = np.random.randint(0, 10, 1000).astype(np.long)
X_test = np.random.rand(100, 784).astype(np.float32)
y_test = np.random.randint(0, 10, 100).astype(np.long)

# Run the PBT
analysis = tune.run(
    train_model,
    name="pbt_test",
    scheduler=scheduler,
    num_samples=4,
    config={
        "lr": tune.loguniform(1e-4, 1e-1),
        "num_epochs": 10,
        "input_size": 784,
        "num_classes": 2,
        "hidden_layers": [128],
        "X_train": X_train,
        "y_train": y_train,
        "X_test": X_test,
        "y_test": y_test
    },
    callbacks=[WandbLoggerCallback(project="project_name", api_key="wandb_api_key")] 
)

best_config = analysis.get_best_config(metric="accuracy", mode="max")
print("Best config:", best_config)
